In [2]:
%cd temp_exps

/temp_exps


In [1]:
import torch as t
from functools import partial
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
from sae_lens import SAE, HookedSAETransformer
from transformer_lens.hook_points import HookPoint
import pandas as pd
import numpy as np
from tqdm import tqdm
from warnings import warn
import os
import time  # For timing the training loops
from functools import partial
from torch import Tensor

import plotly.express as px
update_layout_set = {"xaxis_range", "yaxis_range", "yaxis2_range", "hovermode", "xaxis_title", "yaxis_title", "colorbar", "colorscale", "coloraxis", "title_x", "bargap", "bargroupgap", "xaxis_tickformat", "yaxis_tickformat", "title_y", "legend_title_text", "xaxis_showgrid", "xaxis_gridwidth", "xaxis_gridcolor", "yaxis_showgrid", "yaxis_gridwidth", "yaxis_gridcolor", "showlegend", "xaxis_tickmode", "yaxis_tickmode", "margin", "xaxis_visible", "yaxis_visible", "bargap", "bargroupgap", "xaxis_tickangle"}
def to_numpy(tensor):
    """
    Helper function to convert a tensor to a numpy array. Also works on lists, tuples, and numpy arrays.
    """
    if isinstance(tensor, np.ndarray):
        return tensor
    elif isinstance(tensor, (list, tuple)):
        array = np.array(tensor)
        return array
    elif isinstance(tensor, (Tensor, t.nn.parameter.Parameter)):
        return tensor.detach().cpu().numpy()
    elif isinstance(tensor, (int, float, bool, str)):
        return np.array(tensor)
    else:
        raise ValueError(f"Input to to_numpy has invalid type: {type(tensor)}")
def reorder_list_in_plotly_way(L: list, col_wrap: int):
    '''
    Helper function, because Plotly orders figures in an annoying way when there's column wrap.
    '''
    L_new = []
    while len(L) > 0:
        L_new.extend(L[-col_wrap:])
        L = L[:-col_wrap]
    return L_new


def imshow(tensor, renderer=None, **kwargs):
    kwargs_post = {k: v for k, v in kwargs.items() if k in update_layout_set}
    kwargs_pre = {k: v for k, v in kwargs.items() if k not in update_layout_set}
    if "facet_labels" in kwargs_pre:
        facet_labels = kwargs_pre.pop("facet_labels")
    else:
        facet_labels = None
    if "color_continuous_scale" not in kwargs_pre:
        kwargs_pre["color_continuous_scale"] = "RdBu"
    if "color_continuous_midpoint" not in kwargs_pre:
        kwargs_pre["color_continuous_midpoint"] = -1.6
    if "margin" in kwargs_post and isinstance(kwargs_post["margin"], int):
        kwargs_post["margin"] = dict.fromkeys(list("tblr"), kwargs_post["margin"])
    fig = px.imshow(to_numpy(tensor), **kwargs_pre).update_layout(**kwargs_post)
    if facet_labels:
        # Weird thing where facet col wrap means labels are in wrong order
        if "facet_col_wrap" in kwargs_pre:
            facet_labels = reorder_list_in_plotly_way(facet_labels, kwargs_pre["facet_col_wrap"])
        for i, label in enumerate(facet_labels):
            fig.layout.annotations[i]['text'] = label
    fig.show(renderer=renderer)



/Users/timhua/anaconda3/envs/arena/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/timhua/anaconda3/envs/arena/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <CFED5F8E-EC3F-36FD-AAA3-2C6C7F8D3DD9> /Users/timhua/anaconda3/envs/arena/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <A51C8C05-245A-3989-8D3C-9A6704422CA5> /Users/timhua/anaconda3/envs/arena/lib/python3.11/site-packages/torch/lib/libtorch_cpu.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/Users/timhua/anaconda3/envs/arena/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and to

In [2]:
patch_results = t.load("patch_results.pt")
imshow(
    patch_results,
    labels={"x": "Token Position", "y": "Layer"},
    title="Patching results for the truthfulness information",
    width=1000
)

/var/folders/_s/_vl6g35116x95t0fdlhd31dw0000gn/T/ipykernel_95748/650355143.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  patch_results = t.load("patch_results.pt")


RuntimeError: Attempting to deserialize object on a CUDA device but torch.cuda.is_available() is False. If you are running on a CPU-only machine, please use torch.load with map_location=torch.device('cpu') to map your storages to the CPU.

In [10]:
newp = t.load("trained_probes_truth_twoshot/probe_sae_recons_label_seed_10.pt")

In [11]:
newp

OrderedDict([('net.weight',
              tensor([[ 0.0024, -0.0541, -0.0179,  ...,  0.0254, -0.0362,  0.0615]],
                     device='cuda:0')),
             ('net.bias', tensor([-0.0007], device='cuda:0'))])

In [1]:
import torch as t
import torch.nn as nn
import pandas as pd
import numpy as np
from tqdm import tqdm
from sae_lens import (
    SAE,
    ActivationsStore,
    HookedSAETransformer,
    LanguageModelSAERunnerConfig,
    SAEConfig,
    SAETrainingRunner,
    upload_saes_to_huggingface,
)

In [2]:
?t.optim.AdamW

Init signature:
t.optim.AdamW(
    params: Union[Iterable[torch.Tensor], Iterable[Dict[str, Any]], Iterable[Tuple[str, torch.Tensor]]],
    lr: Union[float, torch.Tensor] = 0.001,
    betas: Tuple[Union[float, torch.Tensor], Union[float, torch.Tensor]] = (0.9, 0.999),
    eps: float = 1e-08,
    weight_decay: float = 0.01,
    amsgrad: bool = False,
    *,
    maximize: bool = False,
    foreach: Optional[bool] = None,
    capturable: bool = False,
    differentiable: bool = False,
    fused: Optional[bool] = None,
)
Docstring:     
Implements AdamW algorithm.

.. math::
   \begin{aligned}
        &\rule{110mm}{0.4pt}                                                                 \\
        &\textbf{input}      : \gamma \text{(lr)}, \: \beta_1, \beta_2
            \text{(betas)}, \: \theta_0 \text{(params)}, \: f(\theta) \text{(objective)},
            \: \epsilon \text{ (epsilon)}                                                    \\
        &\hspace{13mm}      \lambda \text{(weight 

In [2]:
model2b = HookedSAETransformer.from_pretrained("gemma-2-2b", device = 'cuda')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loaded pretrained model gemma-2-2b into HookedTransformer


In [2]:
city_alice = pd.read_csv("cities_alice.csv")
neg_city_alice = pd.read_csv("neg_cities_alice.csv")
data = pd.concat([city_alice, neg_city_alice])
#scramble rows
np.random.seed(123)
data = data.sample(frac = 1).reset_index(drop = True)

In [3]:
data.groupby(['label', 'has_alice', 'has_alice xor label']).size()

label  has_alice  has_alice xor label
0      False      False                  748
       True       True                   748
1      False      True                   748
       True       False                  748
dtype: int64

In [12]:
data.shape

(2992, 11)

In [4]:
model2b.cfg

HookedTransformerConfig:
{'NTK_by_parts_factor': 8.0,
 'NTK_by_parts_high_freq_factor': 4.0,
 'NTK_by_parts_low_freq_factor': 1.0,
 'act_fn': 'gelu_pytorch_tanh',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_scale': 16.0,
 'attn_scores_soft_cap': 50.0,
 'attn_types': ['global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'global',
                'local',
                'globa

In [3]:
sae, cfg_dict, sparsity = SAE.from_pretrained(
    release = "gemma-scope-2b-pt-res-canonical",
    sae_id = "layer_19/width_16k/canonical",
    device = "cuda"
)

In [6]:
a = model2b.generate("Bing bong ding dong", max_new_tokens = 10)
a

  0%|          | 0/10 [00:00<?, ?it/s]

"Bing bong ding dong, I'm blowing this one, 9"

In [5]:
some_batch = model2b.tokenizer(["Ding don bing bong", "King kong ding dong whoop whoop"], return_tensors = "pt", padding = True).to("cuda")

In [6]:
some_batch['attention_mask']

tensor([[1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')

In [7]:
leb = model2b.run_with_cache_with_saes(some_batch['input_ids'],saes = sae)

In [8]:
leb[1].keys()

dict_keys(['hook_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_rot_q', 'blocks.0.attn.hook_rot_k', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.ln1_post.hook_scale', 'blocks.0.ln1_post.hook_normalized', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_pre_linear', 'blocks.0.mlp.hook_post', 'blocks.0.ln2_post.hook_scale', 'blocks.0.ln2_post.hook_normalized', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_rot_q', 'blocks.1.attn.hook_rot_k', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1

In [9]:
leb[1]['blocks.19.hook_resid_post.hook_sae_acts_pre']

tensor([[[ 5.2232e+01, -2.7003e+01,  4.6542e+00,  ..., -5.2398e+01,
           2.0997e+01,  2.1721e-01],
         [-6.7635e+00, -6.1891e+00, -1.1391e+01,  ..., -8.5298e+00,
          -1.1453e+00, -1.9739e+00],
         [-1.6946e+01, -4.8029e+00, -1.9131e+00,  ..., -4.0972e+00,
           3.4327e+00, -5.6123e+00],
         ...,
         [-6.1531e+00, -4.0606e+00, -7.6024e+00,  ..., -2.5864e+00,
          -3.1558e+00, -3.7671e+00],
         [-6.5121e+00, -3.5599e+00, -7.1476e+00,  ..., -2.5776e+00,
          -3.4720e+00, -3.5249e+00],
         [-5.8592e+00, -3.5852e+00, -6.8130e+00,  ..., -3.1476e+00,
          -3.7623e+00, -3.7608e+00]],

        [[ 5.2232e+01, -2.7003e+01,  4.6542e+00,  ..., -5.2398e+01,
           2.0997e+01,  2.1721e-01],
         [-1.6245e+01,  2.8613e+00, -7.4969e+00,  ..., -6.8718e+00,
          -1.0947e+01, -6.6743e+00],
         [-1.6009e+01, -1.0201e+00, -9.5743e+00,  ..., -8.3371e+00,
          -5.1224e+00, -3.5708e+00],
         ...,
         [-2.8493e+01, -7

In [31]:
hmm = leb[1]['blocks.19.hook_resid_post.hook_sae_acts_post'][:,1,:]


In [32]:
for h in hmm:
    print(h)

tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')
tensor([0., 0., 0.,  ..., 0., 0., 0.], device='cuda:0')


In [28]:
leb[1]['blocks.19.hook_resid_post.hook_sae_acts_post'][    1,     0,     263]

tensor(31.0618, device='cuda:0')

In [26]:
t.nonzero(leb[1]['blocks.19.hook_resid_post.hook_sae_acts_post'][:,1:,:])

tensor([[    0,     0,     4],
        [    0,     0,   263],
        [    0,     0,   774],
        ...,
        [    1,     7, 16209],
        [    1,     7, 16303],
        [    1,     7, 16372]], device='cuda:0')

In [12]:
indices = (leb[1]['blocks.19.hook_resid_post.hook_sae_acts_post'] > 0).nonzero(as_tuple=True)[0]


tensor([[[ True, False, False,  ..., False,  True, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]],

        [[ True, False, False,  ..., False,  True, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         ...,
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False],
         [False, False, False,  ..., False, False, False]]], device='cuda:0')

In [ ]:
batch_out = model.run_with_cache_with_saes(
            batch_ids,
            saes=sae,
            names_filter=lambda name: name in [
                'blocks.19.hook_resid_post.hook_sae_input',
                'blocks.19.hook_resid_post.hook_sae_recons'
            ]
        )[1]

In [44]:
leb[1]['blocks.19.hook_resid_post.hook_sae_recons']
# tensor([[[ 6.3655e+01,  2.7575e+02, -1.7398e+02,  ...,  2.9820e+02,
        #   -6.2675e+01,  2.9702e+01],
        #  [-4.8884e+00, -6.8970e-01,  4.2473e-01,  ...,  2.6384e+00,
        #    7.4499e-01,  1.6682e-01],
        #  [-3.9028e+00, -8.6650e-01, -4.0607e+00,  ...,  1.5525e+00,
        #    5.1970e+00, -1.1291e-01],
        #  [-5.4336e+00,  1.4239e+00,  2.3322e+00,  ...,  1.5917e+00,
        #   -5.3451e-01,  2.7083e+00],
        #  [-9.6548e+00,  3.2607e+00,  4.4400e+00,  ..., -2.2930e+00,
        #    1.9895e+00,  2.9883e+00]],

tensor([[[ 6.3655e+01,  2.7575e+02, -1.7398e+02,  ...,  2.9820e+02,
          -6.2675e+01,  2.9702e+01],
         [-4.8884e+00, -6.8970e-01,  4.2473e-01,  ...,  2.6384e+00,
           7.4499e-01,  1.6682e-01],
         [-3.9028e+00, -8.6650e-01, -4.0607e+00,  ...,  1.5525e+00,
           5.1970e+00, -1.1291e-01],
         [-5.4336e+00,  1.4239e+00,  2.3322e+00,  ...,  1.5917e+00,
          -5.3451e-01,  2.7083e+00],
         [-9.6548e+00,  3.2607e+00,  4.4400e+00,  ..., -2.2930e+00,
           1.9895e+00,  2.9883e+00]],

        [[ 6.3655e+01,  2.7575e+02, -1.7398e+02,  ...,  2.9820e+02,
          -6.2675e+01,  2.9702e+01],
         [-6.8911e+00,  8.6366e+00, -2.9912e+00,  ...,  7.0088e-01,
          -1.5728e+00,  1.3779e+00],
         [-4.2630e+00,  1.3513e+00, -2.4401e+00,  ..., -2.6403e+00,
          -4.0709e+00,  3.4806e+00],
         [-3.7446e+00, -6.9691e-01, -8.2747e-01,  ...,  5.4570e+00,
          -1.1321e+00,  3.9979e-01],
         [-6.1591e+00,  2.7186e+00, -5.2874e-01,  ...

In [ ]:
class Probe(nn.Module):
    def __init__(self, activation_dim):
        super().__init__()
        self.net = nn.Linear(activation_dim, 1, bias=True)

    def forward(self, x):
        logits = self.net(x).squeeze(-1)
        return logits

def train_probe(get_acts, label_idx=0, batches=get_data(), lr=1e-2, epochs=1, dim=512, seed=SEED):
    t.manual_seed(seed)
    probe = Probe(dim).to('cuda')
    optimizer = t.optim.AdamW(probe.parameters(), lr=lr)
    criterion = nn.BCEWithLogitsLoss()

    losses = []
    for epoch in range(epochs):
        for batch in tqdm(batches):
            text = batch[0]
            labels = batch[label_idx+1]
            acts = get_acts(text)
            logits = probe(acts)
            loss = criterion(logits, labels.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

    return probe, losses

In [ ]:
#Other people's code

labels = [
    'has_alice',
    'has_not',
    'label',
    'has_alice xor has_not',
    'has_alice xor label',
    'has_not xor label',
    'has_alice xor has_not xor label',
]
    
accs = {}
for label in labels:
    dm = DataManager()
    for dataset in ['cities_alice', 'neg_cities_alice']:
        dm.add_dataset(dataset, 'llama-2-13b-reset', 14, label=label, center=False, split=0.8)
    acts, labels = dm.get('train')
    probe = LRProbe.from_data(acts, labels, bias=True)
    acts, labels = dm.get('val')
    acc = (probe(acts).round() == labels).float().mean()
    accs[label] = acc